<a href="https://colab.research.google.com/github/HosakaKeigo/Google_Colaboratory_repo/blob/main/xlsx_encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## エクセルのパスワードロック
### 使い方
1. driveの指定フォルダにエクセル(.xlsx)を置きます。パスワードはフォルダ名の「~#password.xlsx」のパスワード部分になります。
1. 関数を実行します。（Ctrl + F9）
1. 指定フォルダ配下に`encrypted/`以下にパスワード付きエクセルが生成されます。

### 環境設定
- INPUT_FOLDER_PATH：元ファイルのパス。
- OUTPUT_FOLDER_PATH：パスワードロックしたエクセルファイルの出力先

### 実装について
secure-spreadsheetという、`xlsx-populate`のラッパーライブラリを使っている。

https://github.com/ankane/secure-spreadsheet/tree/master

In [3]:
# secure-spreadsheetの導入
!npm install -g secure-spreadsheet


/tools/node/bin/secure-spreadsheet -> /tools/node/lib/node_modules/secure-spreadsheet/src/cli.js
npm WARN notsup Unsupported engine for commander@11.1.0: wanted: {"node":">=16"} (current: {"node":"14.16.0","npm":"6.14.8"})
npm WARN notsup Not compatible with your version of node/npm: commander@11.1.0

+ secure-spreadsheet@0.2.0
added 22 packages from 44 contributors in 2.639s


In [1]:
# Googleドライブのマウント
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# https://github.com/ankane/secure-spreadsheet
# 特定のディレクトリのxlsxにパスワードをかける
import os
import subprocess
import re

# 出力元、出力先フォルダの指定。
INPUT_FOLDER_PATH = "/content/drive/MyDrive/encrypt"
OUTPUT_FOLDER_PATH = "/content/drive/MyDrive/encrypt/password"

# フォルダを先に作成。
if not os.path.exists(OUTPUT_FOLDER_PATH):
    os.makedirs(OUTPUT_FOLDER_PATH)

# 暗号化
def encrypt_file(input_file_path, output_file_path, password):
    cat_process = subprocess.Popen(["cat", input_file_path], stdout=subprocess.PIPE)

    secure_spreadsheet_process = subprocess.Popen(
        ["secure-spreadsheet", "--password", password, "--input-format", "xlsx"],
        stdin=cat_process.stdout,
        stdout=subprocess.PIPE
    )

    with open(output_file_path, "wb") as f:
        while True:
          # 1024byteで区切る
            data = secure_spreadsheet_process.stdout.read(1024)
            if not data:
                break
            f.write(data)

    cat_process.wait()
    secure_spreadsheet_process.wait()

def validate_filename_format(filename):
    """
    ファイル名は次の規則に従うこと '<filename>#<password>.xlsx'
    """
    return re.match(r'^(.*)#(.*)\.xlsx$', filename) is not None

# 先にファイル規則をassertする。
for filename in os.listdir(INPUT_FOLDER_PATH):
    if filename.endswith('.xlsx') and not validate_filename_format(filename):
        raise ValueError(f"ファイル名「{filename}」が不正です。ファイル名規則： '<filename>#<password>.xlsx'")

for filename in os.listdir(INPUT_FOLDER_PATH):
    if filename.endswith('.xlsx'):
        # ファイル名のうち、「#」以降をパスワードとする
        match = re.match(r'^(.*)#(.*)\.xlsx$', filename)
        if match:
            input_file = os.path.join(INPUT_FOLDER_PATH, filename)
            output_file = os.path.join(OUTPUT_FOLDER_PATH, f"{match.group(1)}.xlsx")
            password = match.group(2)

            print(f"Encrypting {filename} with password {password}")
            encrypt_file(input_file, output_file, password)
            print(f"{filename} encrypted to {output_file}")
        else:
            raise ValueError(f"File {filename} does not match the pattern '<filename>#<password>.xlsx'")


Encrypting 九州（熊本）地区_保存データ#123456789.xlsx with password 123456789
九州（熊本）地区_保存データ#123456789.xlsx encrypted to /content/drive/MyDrive/encrypt/password/九州（熊本）地区_保存データ.xlsx
